## Extracting Data Sources by Report from PBIX layouts for PowerBI
PowerBI files can be quite complex and exceedingly large, making understanding what data sources are being referenced where quite difficult. In the case of handovers between employees, a tool to quickly grasp the overall structure of the pbix file underlying any reporting can be invaluable. Additionally, such a tool is useful for continuous improvement activities such as removing redundant data sources from a dashboard that has evolved over time.

Unfortunately, powerbi does not have any inbuilt functionality to allow such interrogation. However - there is a workaround. Like most "modern" office applications, the ".pbix" file extension is actually a renamed "wrapper" for a ".zip" file that actually contains all the information we need. By interrogating the "Layouts" JSON file in the "Reports" directory of this wrapper, you can piece together a reference table that gives exactly the overview needed.

The below code parses this JSON data (which is otherwise incredibly difficult to read), but requires a few steps to take place first.

    1.**MAKE A COPY** of the .pbix file you want to interrogate
    2. With this **COPY**, rename the ".pbix" extension to ".zip" (AGAIN, ONLY DO THIS TO THE COPY, OTHERWISE YOU WILL BREAK YOUR SOURCE FILE)
    3. Open the .zip file, and go into the "Report" folder within.
    4. Open the "Layout" file in this folder using notepad. (important that you use notepad - other text editors commonly struggle with the text format).
    5. "Save As" the file as a "utf-8" type text document, to a place you'll reference in the code below
    
That's it. Now just change the input and output directory variables in the below code and you'll have a .csv file with the page name, object reference, object type, source of the data used in the object, and how that data is used in the object.

Cheers!

In [1]:
import os
import json
import pandas as pd

In [2]:
#input file paths as necessary
layout_file_path = "Layouts\\Layout UTF8 Example.txt"
export_file_path = 'data_sources.csv'

# EXAMPLE ONLY to look at a subfolder within the same directory you have saved this notebook
## Set a variable for current notebook's path for various loading/saving mechanisms
#nb_path = os.getcwd()
## Specify the path to your json file
#layout_file_path = "{}\\Layouts\\Layout UTF8 Example.txt".format(nb_path)

In [3]:
# import json file
with open(layout_file_path, 'r', encoding='utf-8') as layout_file:
    layout_data = json.load(layout_file)

# Initialize lists to store data
data = []

# Iterate through sections
for section in layout_data.get('sections', []):
    section_name = section.get('displayName', '')
# Iterate through sections
for section in layout_data.get('sections', []):
    section_name = section.get('displayName', '')
    
    # Extract data from filters
    filters_data = json.loads(section.get('filters', '[]'))
    for filter_item in filters_data:
        filter_name = filter_item.get('name', '')
        filter_expression = filter_item.get('expression', {}).get('Column', {}).get('Expression', {}).get('SourceRef', {}).get('Entity', '')
        
        # Append data to the list
        data.append([section_name, filter_name, 'Filter', filter_expression, ''])

    # Extract data from visualContainers
    for visual_container in section.get('visualContainers', []):
        try:
            visual_container_config = json.loads(visual_container.get('config', '{}'))
            visual_container_name = visual_container_config.get('name', '')
            visual_type = visual_container_config.get('singleVisual', {}).get('visualType', '')
            
            # Handle projections attribute
            projections = visual_container_config.get('singleVisual', {}).get('projections', {})
            if isinstance(projections, dict):
                # Handle projections if it's a dictionary
                for projection_type, projection_data in projections.items():
                    for query_ref_data in projection_data:
                        query_ref = query_ref_data.get('queryRef', '')
                        # Append data to the list
                        data.append([section_name, visual_container_name, visual_type, query_ref, projection_type])
            elif isinstance(projections, list):
                # Handle projections if it's a list
                for projection_data in projections:
                    query_ref = projection_data.get('queryRef', '')
                    # Append data to the list
                    data.append([section_name, visual_container_name, visual_type, query_ref, 'UnknownProjectionType'])

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for visual container '{visual_container_name}': {e}")
            continue
        except Exception as e:
            print(f"Error processing visual container '{visual_container_name}': {e}")
            continue

# Create a DataFrame
columns = ['SectionName', 'ObjectName', 'ObjectType', 'QueryRef', 'ProjectionType']
df = pd.DataFrame(data, columns=columns)

# export the DataFrame
df.to_csv(export_file_path, index=False)
